In [3]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import chardet
import pandas as pd
import re
from bs4 import BeautifulSoup
import re


/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import BertTokenizer, BertForMaskedLM, BertPreTrainedModel
from transformers import AlbertForSequenceClassification,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoTokenizer

In [5]:
from spacy import tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = nlp.tokenizer

In [6]:
#Methods

def parse_html(text):
    "removes hyperlink from a piece of text"
    
    soup = BeautifulSoup(text, "html.parser")
    parsed_text = soup.get_text()
    return parsed_text


def spacy_preproc(text):

    "Removes stopwords and punctuation and"

    refined_text = tokenizer(text)
        
    refined_text = [text for text in refined_text if not text.is_stop]
    refined_text = [text.text for text in refined_text if not text.is_punct]
    text = " ".join(refined_text[:])

    return text

def regex_preproc(text): 

    refined_text = re.sub("(\n)", " ", text) #removes python newline 
    refined_text = re.sub("\w+(\')", "" , refined_text) #removes backslash
    refined_text = re.sub("(\s+)", " ", refined_text) #removes whitespaces
    refined_text = re.sub("(&amp|>+|-+)", "", refined_text)
    refined_text = re.sub("(\d+)", "", refined_text) #removes digits
    refined_text = re.sub("\W*(@)", "", refined_text) #removes emails 

    return refined_text
                
def preproc(dict):
    new_dict = {}

    for key,values in dict.items():
        current = []
        
        for value in values:
             
            refined_text = regex_preproc(value)
            refined_text = spacy_preproc(refined_text)
            current.append(refined_text)

        new_dict[key] = current

    #refined_text = re.sub("[>|<-\\", " ", )
    return new_dict


def prepare_corpus(path):
    """BY date"""
    corpus = {}; temp = []; 
    for topic in os.listdir(path):
        subfolder = path + '/' + topic
        current = []
        for doc in os.listdir(subfolder):
            file = subfolder + '/' + doc
            with open(file, 'r', encoding='utf-8', errors= 'ignore') as t:
               #corpus.append((preproc(" ".join(t.readlines())),topic),)
               temp = " ".join(t.readlines())
            current.append(temp)
        corpus[topic] = current
    return corpus

def check_length(text):
    return

def extra_long_documents(text):
    return 

LOADING AND PREPROCESSING

Hyperpartisan

In [7]:

hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Reusing dataset hyperpartisan_news_detection (/Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)
100%|██████████| 2/2 [00:00<00:00, 22.59it/s]


dict_keys(['train', 'validation'])

In [8]:


max_input_length = 512
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=max_input_length)


#change to bypublisher because byarticle has no test set.

hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_test = hyperpartisan_valid.shuffle(seed = 32)
hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)

#
hyperpartisan_train = hyperpartisan_train.map(tokenize, batched=True)
hyperpartisan_valid = hyperpartisan_valid.map(tokenize, batched=True)

hyperpartisan_train = hyperpartisan_train.remove_columns(["title","url","published_at","bias","text"])
hyperpartisan_train = hyperpartisan_train.rename_column("hyperpartisan", "labels")
hyperpartisan_train.set_format('torch')

hyperpartisan_valid = hyperpartisan_valid.remove_columns(["title","url","published_at","bias","text"])
hyperpartisan_valid = hyperpartisan_valid.rename_column("hyperpartisan", "labels")
hyperpartisan_valid.set_format('torch')

print(hyperpartisan_train)
print(hyperpartisan_valid)


Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-31fcbf3d65e8fea2.arrow
Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-66d6117f38d3c868.arrow
Parameter 'function'=<function tokenize at 0x28ec8eca0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
 84%|████████▍ | 503/600 [

OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [ ]:
def tokenize(batch):
#     return tokenizer(batch['review_body'], truncation=True, max_length=max_input_length)
    return tokenizer(batch['sentence'], truncation=True, max_length=max_input_length)
#tokenize dataset

In [ ]:
hyperpartisan_dataset.keys()

dict_keys(['train', 'validation'])

20 Newsgroup

In [4]:
newsgroup_path = "20news-bydate"
element = os.listdir(newsgroup_path)

newsgroup_train_path = newsgroup_path + '/' + element[1]
newsgroup_test_path = newsgroup_path + '/' + element[0]

newsgroup_train = prepare_corpus(newsgroup_train_path)
newsgroup_test = prepare_corpus(newsgroup_test_path)

X_train_newsgroup_text = preproc(newsgroup_train)
X_test_newsgroup_text = preproc(newsgroup_test)

NAIVE BAYES EXPLORATION

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Modelled the conversation with a bag of words approach

In [ ]:
X_train_newsgroup_text.keys()

dict_keys(['talk.politics.mideast', 'rec.autos', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 'comp.os.ms-windows.misc', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'talk.politics.misc', 'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'talk.politics.guns', 'sci.space', 'soc.religion.christian', 'misc.forsale', 'talk.religion.misc'])

In [ ]:
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer

#lemmatize = WordNetLemmatizer()
#stemmer = PorterStemmer()


def extract_features(text, method = "Tfidf"):
    """Count represents the number of features to be chosen from tfidf while text represents text data
    vectorizer is either count_vectorizer or tfidfvectorizer"""
    
    if method.lower() == "tfidf":
        vectorizer = TfidfVectorizer()
    elif method.lower() == "count":
        vectorizer = CountVectorizer()
    else:
        print("Wrong method. Choose between tfidf and count")

    vec = vectorizer.fit_transform(text)
    X_features = vectorizer.get_feature_names()
    print(vec.shape)
    return vec, X_features

def accuracy(prediction, goldstandard):
    acc = 0
    for a,b in zip(prediction,goldstandard):
        print(f"{a}----{b}")
        if a == b:
            acc+=1
        else:pass
    return (acc/len(prediction))*100

In [ ]:
vec, feat = extract_features(X_train_newsgroup_text, "tfidf")
vec.shape

(11314, 105178)


(11314, 105178)

In [31]:
print(feat)

['__' '___' '____' ... 'zzzz' 'zzzzzz' 'zzzzzzt']


In [ ]:
#Initialising multinomial naive bayes
MNB = MultinomialNB()

#explore other forms of naive bayes

MNB.fit(vec[:, :7000],y_label_train)

#extract features from test data
test_vec,test_features = extract_features(X_test_newsgroup_text)

#Used the first 7000 features in making predictions
predicted_topic = MNB.predict(test_vec[:,:7000])
predicted_topic

accuracy(predicted_topic, y_label_test)

(7532, 78590)
rec.motorcycles----talk.politics.mideast
comp.sys.ibm.pc.hardware----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
rec.sport.hockey----talk.politics.mideast
misc.forsale----talk.politics.mideast
comp.os.ms-windows.misc----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
sci.electronics----talk.politics.mideast
comp.sys.mac.hardware----talk.politics.mideast
comp.graphics----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
sci.space----talk.politics.mideast
comp.windows.x----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
rec.sport.hockey----talk.politics.mideast
misc.forsale----talk.politic

/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


4.500796601168348

SVM

In [ ]:
from sklearn import svm

svc_classifier = svm.SVC(decision_function_shape='ovo')
svc_classifier.fit(vec[:,:7000],y_label_train)
svc_prediction = svc_classifier.predict(test_vec[:,:7000])

accuracy(svc_prediction, y_label_test)

misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale

5.177907594264472

BERT fine-tune 

In [ ]:
labels = hyperpartisan_train["text"]
len(labels) #len(hyperpartisan_train["label"])

NameError: name 'hyperpartisan_train' is not defined

In [59]:
from datasets import load_metric
import numpy as np

labels = hyperpartisan_train["label"]
#num_labels = len(labels)
#labels2id, id2labels = dict(),dict()
#for i,label in enumerate(labels):
    #labels2id[label] = i
    #id2labels[i] = label
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions = predictions, references = label)

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#default training arguments 
training_args = TrainingArguments(output_dir="./results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)

trainer = Trainer(model = model, 
            args = training_args,
            train_dataset= hyperpartisan_train,
            eval_dataset= hyperpartisan_valid,
            tokenizer= tokenizer,
            data_collator=data_collator,
            compute_metrics= compute_metrics)

trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/max/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /Users/max/.cache/huggingface/transformers/9c1

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label', 'attention_mask']